In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme.models import loglogit, piecewise_formula, boxcox, lognested
from biogeme.nests import OneNestForNestedLogit, NestsForNestedLogit
from scipy.stats import chi2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [2]:
# Loading the data
df = pd.read_csv('lpmc06.dat', sep='\t')

In [3]:
df["dur_pt"] = df["dur_pt_access"] + df["dur_pt_rail"] + df["dur_pt_bus"] + df["dur_pt_int"]
df["cost_driving"] = df["cost_driving_fuel"] + df["cost_driving_ccharge"]

In [4]:
display(df.columns)

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent', 'dur_pt',
       'cost_driving'],
      dtype='object')

In [5]:
display(df.head())

trip_id  household_id  person_n  trip_n  travel_mode  purpose  fueltype  \
0        7             0         2       1            4        3         1   
1       21             5         1       1            4        3         1   
2       27             7         1       0            4        4         2   
3       52            12         1       2            4        5         2   
4       53            12         1       3            4        3         2   

   faretype  bus_scale  survey_year  ...  dur_pt_bus  dur_pt_int  \
0         3        0.0            1  ...    0.055556    0.000000   
1         5        0.0            1  ...    0.293611    0.000000   
2         1        1.0            1  ...    0.000000    0.100000   
3         1        1.0            1  ...    0.491944    0.094722   
4         1        1.0            1  ...    0.061944    0.000000   

   pt_interchanges  dur_driving  cost_transit  cost_driving_fuel  \
0                0     0.059444           0.0               0.15   
1                0     0.167778           0.0               0.46   
2                1     0.340833           1.5               2.04   
3                1     0.355556           3.0               1.19   
4                0     0.062500           1.5               0.17   

   cost_driving_ccharge  driving_traffic_percent    dur_pt  cost_driving  
0                   0.0                 0.112150  0.165000          0.15  
1                   0.0                 0.243377  0.377500          0.46  
2                   0.0                 0.280359  0.810556          2.04  
3                   0.0                 0.249219  0.649167          1.19  
4                   0.0                 0.124444  0.144444          0.17  

[5 rows x 34 columns]

In [6]:
database = db.Database('lpmc', df)

## Model 0

We identify the variables that will enter the model specification.

In [7]:
# Time related variables
dur_walking = Variable('dur_walking') # in hours
dur_cycling = Variable('dur_cycling') # in hours
# dur_pt_access = Variable('dur_pt_access') # in hours
# dur_pt_rail = Variable('dur_pt_rail') # in hours
# dur_pt_bus = Variable('dur_pt_bus') # in hours
# dur_pt_int = Variable('dur_pt_int') # in hours
dur_pt = Variable('dur_pt')
dur_driving = Variable('dur_driving') # in hours


# Cost related variables
cost_transit = Variable('cost_transit') # in GBP
# cost_driving_fuel = Variable('cost_driving_fuel') # in GBP
# cost_driving_ccharge = Variable('cost_driving_ccharge') # in GBP
cost_driving = Variable('cost_driving')

# Choice taken by the individual
travel_mode = Variable('travel_mode') # 1 = walk, 2 = cycle, 3 = PT, 4 = car

Parameters to be estimated

In [8]:
# ASC_WALK = Beta('asc_walk', 0, None, None, 0)
ASC_CYCLE = Beta('asc_cycle', 0, None, None, 0)
ASC_PT = Beta('asc_pt', 0, None, None, 0)
ASC_CAR = Beta('asc_car', 0, None, None, 0)

B_TIME = Beta('b_time', 0, None, None, 0)
B_COST = Beta('b_cost', 0, None, None, 0)

Definition of the utility functions.

In [9]:
# Walk
V1 = (
  # ASC_WALK -> Normalized with respect to walk
  B_TIME * dur_walking
)

# Cycle
V2 = (
  ASC_CYCLE
  + B_TIME * dur_cycling
)

# Public transport
V3 = (
  ASC_PT
  + B_TIME * dur_pt
  #+ B_TIME * (dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int)
  + B_COST * cost_transit
)

# Car
V4 = (
  ASC_CAR
  + B_TIME * dur_driving
  + B_COST * cost_driving
  #+ B_COST * (cost_driving_fuel + cost_driving_ccharge)
)

In [10]:
V = {1: V1, 2: V2, 3: V3, 4: V4}

Definition of the model.

In [11]:
# All alternatives are available to all individuals.
logprob = loglogit(V, None, travel_mode)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_0'

Estimate the parameters.

In [12]:
results = biogeme.estimate()

Summary statistics.

In [13]:
print(results.print_general_statistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4658.601
Final log likelihood:	-4658.601
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00107
Akaike Information Criterion:	9327.202
Bayesian Information Criterion:	9359.788
Final gradient norm:	1.0183E-03
Nbr of threads:	8



In [14]:
display(results.get_estimated_parameters())

Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_car   -1.264832      0.078562   -16.099815           0.0
asc_cycle -3.742817      0.103011   -36.334033           0.0
asc_pt    -0.552210      0.053870   -10.250839           0.0
b_cost    -0.160728      0.014288   -11.248889           0.0
b_time    -5.340745      0.197809   -26.999514           0.0

## Model 1
In this version of the model, we introduce alternative-specific coefficients for either b_time (model 1a) or b_cost (model 1b), and we compare the results to determine which model yields the best results 

### Model 1a (alternative-specific b_time)


Parameters to be estimated

In [15]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_1A = Beta('ASC_CYCLE_1A', 0, None, None, 0)
ASC_PT_1A    = Beta('ASC_PT_1A', 0, None, None, 0)
ASC_CAR_1A   = Beta('ASC_CAR_1A', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_1A  = Beta('B_TIME_WALK_1A', 0, None, None, 0)
B_TIME_CYCLE_1A = Beta('B_TIME_CYCLE_1A', 0, None, None, 0)
B_TIME_PT_1A    = Beta('B_TIME_PT_1A', 0, None, None, 0)
B_TIME_CAR_1A   = Beta('B_TIME_CAR_1A', 0, None, None, 0)

# Generic cost coefficient
B_COST_1A = Beta('B_COST_1A', 0, None, None, 0)

Definition of the utility functions

In [16]:
V_walk_1A = (
    B_TIME_WALK_1A * dur_walking
)

V_cycle_1A = (
    ASC_CYCLE_1A
    + B_TIME_CYCLE_1A * dur_cycling
)

V_PT_1A = (
    ASC_PT_1A
    + B_TIME_PT_1A * dur_pt
    + B_COST_1A * cost_transit
)

V_car_1A = (
    ASC_CAR_1A
    + B_TIME_CAR_1A * dur_driving
    + B_COST_1A * cost_driving
)

V_1A = {1: V_walk_1A, 2: V_cycle_1A, 3: V_PT_1A, 4: V_car_1A}

Definition of the model

In [17]:
logprob_1A = loglogit(V_1A, None, travel_mode)
biogeme_1A = bio.BIOGEME(database, logprob_1A)
biogeme_1A.modelName = 'model_1A'

Estimation and display of the results

In [18]:
results_1A = biogeme_1A.estimate()
print(results_1A.print_general_statistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4319.913
Final log likelihood:	-4319.913
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00185
Akaike Information Criterion:	8655.826
Bayesian Information Criterion:	8707.963
Final gradient norm:	9.2739E-03
Nbr of threads:	8



In [19]:
display(results_1A.get_estimated_parameters())

Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_1A      -2.087660      0.133307   -15.660566           0.0
ASC_CYCLE_1A    -4.799358      0.193220   -24.838868           0.0
ASC_PT_1A       -2.552549      0.135051   -18.900568           0.0
B_COST_1A       -0.140590      0.015325    -9.174014           0.0
B_TIME_CAR_1A   -6.120530      0.405937   -15.077542           0.0
B_TIME_CYCLE_1A -5.197150      0.467915   -11.107040           0.0
B_TIME_PT_1A    -3.256306      0.254676   -12.786052           0.0
B_TIME_WALK_1A  -8.643431      0.418678   -20.644601           0.0

_Observations for the report_

First, we observe that all the B coefficients are negative, which makes sense, because a longer and/or more expensive travel mode is less attractive. 

The cost coefficient of model 1A is very close to the one for the model 0. On the other hand, the time coefficient, which was made alternative specific in model 1A, now strongly depends on the chosen mode. In particular, we observe it is the smallest (in absolute value) for PT; an interpretation could be that commuters are more prone to long PT travel times because they can read, sleep, etc., activites they cannot do while driving or cycling. On the other hand, the largest time_coefficient (again, in absolute value) is for walking, probably because beyond 20-30 minutes, people consider that doing the route by foot is too long.

If we compare the Akaike or Bayesian information criterion between model 0 and model 1a, we observe they are both lower in the case of model 1a; it means the latter fits the data better than model 0.

### Model 1b (alternative-specific b_cost)

We reproduce exactly the same steps as for model 1A, but we now assume a generic time coefficient, and a alternative specific cost coefficient

Parameters to be estimated

In [20]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_1B = Beta('ASC_CYCLE_1B', 0, None, None, 0)
ASC_PT_1B    = Beta('ASC_PT_1B', 0, None, None, 0)
ASC_CAR_1B   = Beta('ASC_CAR_1B', 0, None, None, 0)

# Alternative specific cost coefficient
B_COST_PT_1B    = Beta('B_COST_PT_1B', 0, None, None, 0)
B_COST_CAR_1B   = Beta('B_COST_CAR_1B', 0, None, None, 0)

# Generic time coefficient
B_TIME_1B = Beta('B_TIME_1B', 0, None, None, 0)

Definition of the utility functions 

In [21]:
V_walk_1B = (
    B_TIME_1B * dur_walking
)

V_cycle_1B = (
    ASC_CYCLE_1B
    + B_TIME_1B * dur_cycling
)

V_PT_1B = (
    ASC_PT_1B
    + B_TIME_1B * dur_pt
    + B_COST_PT_1B * cost_transit
)

V_car_1B = (
    ASC_CAR_1B
    + B_TIME_1B * dur_driving
    + B_COST_CAR_1B * cost_driving
)

V_1B = {1: V_walk_1B, 2: V_cycle_1B, 3: V_PT_1B, 4: V_car_1B}

Definition of the model

In [22]:
logprob_1B = loglogit(V_1B, None, travel_mode)
biogeme_1B = bio.BIOGEME(database, logprob_1B)
biogeme_1B.modelName = 'model_1B'

Estimation and display of the results

In [23]:
results_1B = biogeme_1B.estimate()
print(results_1B.print_general_statistics())

Number of estimated parameters:	6
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4607.516
Final log likelihood:	-4607.516
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.0013
Akaike Information Criterion:	9227.032
Bayesian Information Criterion:	9266.136
Final gradient norm:	5.3980E-04
Nbr of threads:	8



In [24]:
display(results_1B.get_estimated_parameters())

Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_1B    -1.181056      0.078388   -15.066824      0.000000
ASC_CYCLE_1B  -3.698346      0.103376   -35.775502      0.000000
ASC_PT_1B     -0.836648      0.065575   -12.758696      0.000000
B_COST_CAR_1B -0.205583      0.021929    -9.374982      0.000000
B_COST_PT_1B   0.052828      0.029809     1.772231      0.076356
B_TIME_1B     -5.339547      0.200044   -26.691801      0.000000

In [25]:
lr_result1A = results_1A.likelihood_ratio_test(results, 0.05)
print(f'{lr_result1A.statistic=:.3g}')
print(f'{lr_result1A.threshold=:.3g}')
print(lr_result1A.message)

lr_result1A.statistic=677
lr_result1A.threshold=7.81
H0 can be rejected at level 5.0%


In [26]:
lr_result1B = results_1B.likelihood_ratio_test(results, 0.05)
print(f'{lr_result1B.statistic=:.3g}')
print(f'{lr_result1B.threshold=:.3g}')
print(lr_result1B.message)

lr_result1B.statistic=102
lr_result1B.threshold=3.84
H0 can be rejected at level 5.0%


_Observations for the report_

This model seems clearly less effective than model 1A. We see it because

1) The Bayesian and Akaike criterion are higher for model 1B
2) the t-test are closer to 0
3) the cost coefficient for PT is higher than 0, which doesn't make much sense

We could have guessed that model 1B would be less interesting than model 1A, because only two alternatives out of four have a cost parameter, which means that making B_cost alternative specific allows less flexibility in the model than making B_time alternative specific

So our preferred model for the rest of the project will be model 1A (alternative specific time coefficients)

To formalize this, we perform a Cox test. We define a general model containing both an alternative-specific b_time and alt-spec b_cost

In [27]:
# Utility functions
V_walk_1Cox = (B_TIME_WALK_1A * dur_walking)

V_cycle_1Cox = (
    ASC_CYCLE
    + B_TIME_CYCLE_1A * dur_cycling
)

V_PT_1Cox = (
    ASC_PT
    + B_TIME_PT_1A * dur_pt
    + B_COST_PT_1B * cost_transit
)

V_car_1Cox = (
    ASC_CAR
    + B_TIME_CAR_1A * dur_driving
    + B_COST_CAR_1B * cost_driving
)

V_1Cox = {1: V_walk_1Cox, 2: V_cycle_1Cox, 3: V_PT_1Cox, 4: V_car_1Cox}

# Estimation of the results
logprob_1Cox = loglogit(V_1Cox, None, travel_mode)
biogeme_1Cox = bio.BIOGEME(database, logprob_1Cox)
biogeme_1Cox.modelName = 'model_1Cox'
results_1Cox = biogeme_1Cox.estimate()

We compare model 1C with model 1A

In [28]:
lr_result1AC = results_1A.likelihood_ratio_test(results_1Cox, 0.005)
print(f'{lr_result1AC.statistic=:.3g}')
print(f'{lr_result1AC.threshold=:.3g}')
print(lr_result1AC.message)

lr_result1AC.statistic=7.03
lr_result1AC.threshold=7.88
H0 cannot be rejected at level 0.5%


We compare model 1C with model 1B

In [29]:
lr_result1BC = results_1B.likelihood_ratio_test(results_1Cox, 0.005)
print(f'{lr_result1BC.statistic=:.3g}')
print(f'{lr_result1BC.threshold=:.3g}')
print(lr_result1BC.message)

lr_result1BC.statistic=582
lr_result1BC.threshold=12.8
H0 can be rejected at level 0.5%


In [30]:
V_pref = V_1A
results_pref = results_1A

To conclude, model 1A is better than model 1B because, at the level $\alpha = 0.5 \%$, model B can be rejected against the composite model, but model A cannot. By the Cox test, 1A is better than 1B

## Model 2

#### Age

In [31]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_2A = Beta('ASC_CYCLE_2A', 0, None, None, 0)
ASC_PT_2A    = Beta('ASC_PT_2A', 0, None, None, 0)
ASC_CAR_2A   = Beta('ASC_CAR_2A', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_2A  = Beta('B_TIME_WALK_2A', 0, None, None, 0)
B_TIME_CYCLE_2A = Beta('B_TIME_CYCLE_2A', 0, None, None, 0)
B_TIME_PT_2A    = Beta('B_TIME_PT_2A', 0, None, None, 0)
B_TIME_CAR_2A   = Beta('B_TIME_CAR_2A', 0, None, None, 0)

# Generic cost coefficient
B_COST_2A = Beta('B_COST_2A', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_2A = Beta('B_AGE_CYCLE_2A', 0, None, None, 0)
B_AGE_PT_2A = Beta('B_AGE_PT_2A', 0, None, None, 0)
B_AGE_CAR_2A = Beta('B_AGE_CAR_2A', 0, None, None, 0)

In [32]:
V_walk_2A = (
    B_TIME_WALK_2A * dur_walking
)

V_cycle_2A = (
    ASC_CYCLE_2A
    + B_AGE_CYCLE_2A * age
    + B_TIME_CYCLE_2A * dur_cycling
)

V_pt_2A = (
    ASC_PT_2A
    + B_AGE_PT_2A * age
    + B_TIME_PT_2A * dur_pt
    + B_COST_2A * cost_transit
)

V_car_2A = (
    ASC_CAR_2A
    + B_AGE_CAR_2A * age
    + B_TIME_CAR_2A * dur_driving
    + B_COST_2A * cost_driving
)

V_2A ={1: V_walk_2A, 2: V_cycle_2A, 3: V_pt_2A, 4: V_car_2A}

logprob_2A = loglogit(V_2A, None, travel_mode)
biogeme_2A = bio.BIOGEME(database, logprob_2A)
biogeme_2A.modelName = 'model_2A'

results_2A = biogeme_2A.estimate()
print(results_2A.print_general_statistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4285.994
Final log likelihood:	-4285.994
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00257
Akaike Information Criterion:	8593.988
Bayesian Information Criterion:	8665.678
Final gradient norm:	1.2485E-01
Nbr of threads:	8



In [33]:
display(results_2A.get_estimated_parameters())

Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_2A      -2.875254      0.176813   -16.261510  0.000000e+00
ASC_CYCLE_2A    -4.943190      0.262985   -18.796450  0.000000e+00
ASC_PT_2A       -3.058113      0.175231   -17.451942  0.000000e+00
B_AGE_CAR_2A     0.018770      0.002321     8.087943  6.661338e-16
B_AGE_CYCLE_2A   0.002596      0.004206     0.617245  5.370734e-01
B_AGE_PT_2A      0.012036      0.002509     4.796941  1.611073e-06
B_COST_2A       -0.141627      0.015358    -9.221744  0.000000e+00
B_TIME_CAR_2A   -6.067418      0.404419   -15.002793  0.000000e+00
B_TIME_CYCLE_2A -5.229308      0.472931   -11.057242  0.000000e+00
B_TIME_PT_2A    -3.249322      0.253367   -12.824576  0.000000e+00
B_TIME_WALK_2A  -8.763746      0.430897   -20.338364  0.000000e+00

In [34]:
lr_result2A = results_2A.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result2A.statistic=:.3g}')
print(f'{lr_result2A.threshold=:.3g}')
print(lr_result2A.message)

lr_result2A.statistic=67.8
lr_result2A.threshold=7.81
H0 can be rejected at level 5.0%


In [35]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_2B = Beta('ASC_CYCLE_2B', 0, None, None, 0)
ASC_PT_2B    = Beta('ASC_PT_2B', 0, None, None, 0)
ASC_CAR_2B   = Beta('ASC_CAR_2B', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_2B  = Beta('B_TIME_WALK_2B', 0, None, None, 0)
B_TIME_CYCLE_2B = Beta('B_TIME_CYCLE_2B', 0, None, None, 0)
B_TIME_PT_2B    = Beta('B_TIME_PT_2B', 0, None, None, 0)
B_TIME_CAR_2B   = Beta('B_TIME_CAR_2B', 0, None, None, 0)

# Generic cost coefficient
B_COST_2B = Beta('B_COST_2B', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_2B = Beta('B_AGE_CYCLE_2B', 0, None, None, 0)
B_AGE_PT_2B = Beta('B_AGE_PT_2B', 0, None, None, 0)
B_AGE_CAR_2B = Beta('B_AGE_CAR_2B', 0, None, None, 0)

In [36]:
V_walk_2B = (
    B_TIME_WALK_2B * dur_walking 
)

V_cycle_2B = (
    ASC_CYCLE_2B
    + B_TIME_CYCLE_2B * dur_cycling 
)

V_PT_2B = (
    ASC_PT_2B
    + B_TIME_PT_2B * dur_pt 
    + B_COST_2B * cost_transit * age
)

V_car_2B = (
    ASC_CAR_2B
    + B_TIME_CAR_2B * dur_driving 
    + B_COST_2B * cost_driving * age
)

V_2B ={1: V_walk_2B, 2: V_cycle_2B, 3: V_PT_2B, 4: V_car_2B}

logprob_2B = loglogit(V_2B, None, travel_mode)
biogeme_2B = bio.BIOGEME(database, logprob_2B)
biogeme_2B.modelName = 'model_2B'

results_2B = biogeme_2B.estimate()
print(results_2B.print_general_statistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4332.703
Final log likelihood:	-4332.703
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00185
Akaike Information Criterion:	8681.407
Bayesian Information Criterion:	8733.544
Final gradient norm:	4.2487E-01
Nbr of threads:	8



In [37]:
display(results_2B.get_estimated_parameters())

Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_2B      -2.093516      0.133448   -15.687844  0.000000e+00
ASC_CYCLE_2B    -4.810449      0.193347   -24.879912  0.000000e+00
ASC_PT_2B       -2.561539      0.134996   -18.974871  0.000000e+00
B_COST_2B       -0.003017      0.000445    -6.780019  1.201594e-11
B_TIME_CAR_2B   -6.260847      0.403183   -15.528552  0.000000e+00
B_TIME_CYCLE_2B -5.186415      0.467821   -11.086319  0.000000e+00
B_TIME_PT_2B    -3.334402      0.253651   -13.145645  0.000000e+00
B_TIME_WALK_2B  -8.658976      0.418431   -20.693918  0.000000e+00

In [38]:
lr_result2B = results_2B.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result2B.statistic=:.3g}')
print(f'{lr_result2B.threshold=:.3g}')
print(lr_result2B.message)

lr_result2B.statistic=25.6
lr_result2B.threshold=nan
H0 can be rejected at level 5.0%


In [39]:
V_pref = V_2A
results_pref = results_2A

## Model 3

Using Model_pref as the base model, include an appropriate non-linear transformation of one of the variables. Report both the specication and the estimation results (as defined previously).

In [40]:
sns.histplot(df["dur_walking"])
sns.histplot(df["dur_cycling"])
sns.histplot(df["dur_pt_access"] + df["dur_pt_rail"] + df["dur_pt_bus"] + df["dur_pt_int"])
sns.histplot(df["dur_driving"])
plt.xscale('log')
plt.show()

#### Piecewise Linear

In [41]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3A = Beta('ASC_CYCLE_3A', 0, None, None, 0)
ASC_PT_3A    = Beta('ASC_PT_3A', 0, None, None, 0)
ASC_CAR_3A   = Beta('ASC_CAR_3A', 0, None, None, 0)

# Generic cost coefficient
B_COST_3A = Beta('B_COST_3A', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_3A = Beta('B_AGE_CYCLE_3A', 0, None, None, 0)
B_AGE_PT_3A = Beta('B_AGE_PT_3A', 0, None, None, 0)
B_AGE_CAR_3A = Beta('B_AGE_CAR_3A', 0, None, None, 0)

In [42]:
thresholds = [0, 0.1, 0.5, 1, 2, None]

BETA_TIME_3A_1 = Beta('BETA_TIME_3A_0_0.1', 0, None, None, 0)
BETA_TIME_3A_2 = Beta('BETA_TIME_3A_0.1_0.5', 0, None, None, 0)
BETA_TIME_3A_3 = Beta('BETA_TIME_3A_0.5_1', 0, None, None, 0)
BETA_TIME_3A_4 = Beta('BETA_TIME_3A_1_2', 0, None, None, 0)
BETA_TIME_3A_5 = Beta('BETA_TIME_3A_2_more', 0, None, None, 0)
betas_piecewise = [BETA_TIME_3A_1, BETA_TIME_3A_2, BETA_TIME_3A_3, BETA_TIME_3A_4, BETA_TIME_3A_5]

In [43]:
piecewise_dur_walking = piecewise_formula(
    dur_walking, thresholds, betas_piecewise
)
piecewise_dur_cycling = piecewise_formula(
    dur_cycling, thresholds, betas_piecewise
)
piecewise_dur_pt = piecewise_formula(
    dur_pt, thresholds, betas_piecewise
)
piecewise_dur_driving = piecewise_formula(
    dur_driving, thresholds, betas_piecewise
)

In [44]:
V_walk_3A_piecewise = (
    piecewise_dur_walking
)

V_cycle_3A_piecewise = (
    ASC_CYCLE_3A
    + B_AGE_CYCLE_3A * age
    + piecewise_dur_cycling
)

V_pt_3A_piecewise = (
    ASC_PT_3A
    + B_AGE_PT_3A * age
    + piecewise_dur_pt
    + B_COST_3A * cost_transit
)

V_car_3A_piecewise = (
    ASC_CAR_3A
    + B_AGE_CAR_3A * age
    + piecewise_dur_driving
    + B_COST_3A * cost_driving
)

V_3A_piecewise ={1: V_walk_3A_piecewise, 2: V_cycle_3A_piecewise, 3: V_pt_3A_piecewise, 4: V_car_3A_piecewise}
logprob_3A_piecewise = loglogit(V_3A_piecewise, None, travel_mode)
biogeme_3A_piecewise = bio.BIOGEME(database, logprob_3A_piecewise)
biogeme_3A_piecewise.modelName = 'model_3A_piecewise'

results_3A_piecewise = biogeme_3A_piecewise.estimate()
print(results_3A_piecewise.print_general_statistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4472.032
Final log likelihood:	-4472.032
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00268
Akaike Information Criterion:	8968.064
Bayesian Information Criterion:	9046.27
Final gradient norm:	4.3981E-01
Nbr of threads:	8



In [45]:
results_3A_piecewise.getEstimatedParameters()

C:\Users\vaffs\AppData\Local\Temp\ipykernel_22552\159394502.py:1: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_3A_piecewise.getEstimatedParameters()


Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_3A            -3.065167      0.142653   -21.486876  0.000000e+00
ASC_CYCLE_3A          -4.718680      0.194083   -24.312658  0.000000e+00
ASC_PT_3A             -1.431023      0.108573   -13.180246  0.000000e+00
BETA_TIME_3A_0.1_0.5 -10.577666      0.390148   -27.111960  0.000000e+00
BETA_TIME_3A_0.5_1    -4.056305      0.232834   -17.421470  0.000000e+00
BETA_TIME_3A_0_0.1   -16.802503      2.377475    -7.067374  1.578959e-12
BETA_TIME_3A_1_2      -4.988176      0.409427   -12.183300  0.000000e+00
BETA_TIME_3A_2_more   -3.720641      1.831126    -2.031887  4.216513e-02
B_AGE_CAR_3A           0.017506      0.002193     7.982032  1.332268e-15
B_AGE_CYCLE_3A         0.000719      0.003977     0.180790  8.565323e-01
B_AGE_PT_3A            0.008808      0.002203     3.998606  6.371666e-05
B_COST_3A             -0.127327      0.015265    -8.340927  0.000000e+00

In [46]:
piecewise_estimates = results_3A_piecewise.get_beta_values()
V_walk_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_cycle_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_pt_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')
V_car_3A_piecewise.fix_betas(beta_values=piecewise_estimates, prefix='piecewise_')

In [47]:
alpha = Beta('alpha', 0, None, None, 0)
Opt1 = (1 - alpha) * V_walk_2A + alpha * V_walk_3A_piecewise
Opt2 = (1 - alpha) * V_cycle_2A + alpha * V_cycle_3A_piecewise
Opt3 = (1 - alpha) * V_pt_2A + alpha * V_pt_3A_piecewise
Opt4 = (1 - alpha) * V_car_2A + alpha * V_car_3A_piecewise

V = {1: Opt1, 2: Opt2, 3: Opt3, 4: Opt4}
logprob = loglogit(V, None, travel_mode)
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'model_3A_composite'
results_composite = biogeme.estimate()


In [48]:
composite_parameters = results_composite.get_estimated_parameters()
display(composite_parameters)

Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_2A       -4.196995      0.583947    -7.187291  6.608047e-13
ASC_CYCLE_2A     -6.533354      0.792531    -8.243660  2.220446e-16
ASC_PT_2A        -5.671933      0.927827    -6.113135  9.769274e-10
B_AGE_CAR_2A      0.022671      0.006294     3.602180  3.155601e-04
B_AGE_CYCLE_2A    0.007373      0.010956     0.672970  5.009666e-01
B_AGE_PT_2A       0.017974      0.006788     2.647778  8.102279e-03
B_COST_2A        -0.164077      0.040582    -4.043149  5.273808e-05
B_TIME_CAR_2A    -3.530501      1.273989    -2.771218  5.584700e-03
B_TIME_CYCLE_2A  -2.562154      1.357838    -1.886936  5.916897e-02
B_TIME_PT_2A      0.878168      1.505368     0.583358  5.596524e-01
B_TIME_WALK_2A  -11.786056      1.244758    -9.468551  0.000000e+00
alpha             0.611788      0.078938     7.750207  9.103829e-15

In [49]:
t_test_alpha = composite_parameters.loc['alpha', 'Rob. t-test']
print(f't-test for alpha: {t_test_alpha:.3g}')

t-test for alpha: 7.75


In [50]:
p_value_alpha = composite_parameters.loc['alpha', 'Rob. p-value']
print(f'p-value for alpha: {p_value_alpha:.3g}')


p-value for alpha: 9.1e-15


Under the null hypothesis, the true value of $\alpha$ is 0. This hypothesis can be safely rejected here. Therefore, the piecewise linear specification is preferred.

#### Box-Cox

In [51]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3B = Beta('ASC_CYCLE_3B', 0, None, None, 0)
ASC_PT_3B    = Beta('ASC_PT_3B', 0, None, None, 0)
ASC_CAR_3B   = Beta('ASC_CAR_3B', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3B  = Beta('B_TIME_WALK_3B', 0, None, None, 0)
B_TIME_CYCLE_3B = Beta('B_TIME_CYCLE_3B', 0, None, None, 0)
B_TIME_PT_3B    = Beta('B_TIME_PT_3B', 0, None, None, 0)
B_TIME_CAR_3B   = Beta('B_TIME_CAR_3B', 0, None, None, 0)

# Generic cost coefficient
B_COST_3B = Beta('B_COST_3B', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_3B = Beta('B_AGE_CYCLE_3B', 0, None, None, 0)
B_AGE_PT_3B = Beta('B_AGE_PT_3B', 0, None, None, 0)
B_AGE_CAR_3B = Beta('B_AGE_CAR_3B', 0, None, None, 0)

In [52]:
lambda_boxcox_walking = Beta('lambda_boxcox_walking', 1, -10, 10, 0)
boxcox_dur_walking_unrestricted = boxcox(dur_walking, lambda_boxcox_walking)

lambda_boxcox_cycling = Beta('lambda_boxcox_cycling', 1, -10, 10, 0)
boxcox_dur_cycling_unrestricted = boxcox(dur_cycling, lambda_boxcox_cycling)

lambda_boxcox_pt = Beta('lambda_boxcox_pt', 1, -10, 10, 0)
boxcox_dur_pt_unrestricted = boxcox(dur_pt, lambda_boxcox_pt)

lambda_boxcox_driving = Beta('lambda_boxcox_driving', 1, -10, 10, 0)
boxcox_dur_driving_unrestricted = boxcox(dur_driving, lambda_boxcox_driving)

In [53]:
V_walk_3B_unrestricted = (
    B_TIME_WALK_3B * boxcox_dur_walking_unrestricted
)

V_cycle_3B_unrestricted = (
    ASC_CYCLE_3B
    + B_AGE_CYCLE_3B * age
    + B_TIME_CYCLE_3B * boxcox_dur_cycling_unrestricted
)

V_pt_3B_unrestricted = (
    ASC_PT_3B
    + B_AGE_PT_3B * age
    + B_TIME_PT_3B * boxcox_dur_pt_unrestricted
    + B_COST_3B * cost_transit
)

V_car_3B_unrestricted = (
    ASC_CAR_3B
    + B_AGE_CAR_3B * age
    + B_TIME_CAR_3B * boxcox_dur_driving_unrestricted
    + B_COST_3B * cost_driving
)

V_3B_unrestricted ={1: V_walk_3B_unrestricted, 2: V_cycle_3B_unrestricted, 3: V_pt_3B_unrestricted, 4: V_car_3B_unrestricted}
logprob_3B_unrestricted = loglogit(V_3B_unrestricted, None, travel_mode)
biogeme_3B_unrestricted = bio.BIOGEME(database, logprob_3B_unrestricted)
biogeme_3B_unrestricted.modelName = 'model_3B_unrestricted'

results_3B_unrestricted = biogeme_3B_unrestricted.estimate()
print(results_3B_unrestricted.print_general_statistics())

Number of estimated parameters:	15
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4172.306
Final log likelihood:	-4172.306
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.0036
Akaike Information Criterion:	8374.613
Bayesian Information Criterion:	8472.371
Final gradient norm:	4.2296E-01
Nbr of threads:	8



In [54]:
results_3B_unrestricted.getEstimatedParameters()

C:\Users\vaffs\AppData\Local\Temp\ipykernel_22552\2375549392.py:1: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_3B_unrestricted.getEstimatedParameters()


Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_3B            -1.020518      0.270910    -3.766997  1.652227e-04
ASC_CYCLE_3B          -2.521551      0.318053    -7.928077  2.220446e-15
ASC_PT_3B              0.825456      0.210230     3.926435  8.621431e-05
B_AGE_CAR_3B           0.020680      0.002534     8.159368  4.440892e-16
B_AGE_CYCLE_3B         0.004152      0.004227     0.982324  3.259402e-01
B_AGE_PT_3B            0.014511      0.002829     5.129823  2.900154e-07
B_COST_3B             -0.127028      0.015436    -8.229409  2.220446e-16
B_TIME_CAR_3B         -3.409030      0.302809   -11.258020  0.000000e+00
B_TIME_CYCLE_3B       -3.054685      0.413442    -7.388429  1.485478e-13
B_TIME_PT_3B          -3.214257      0.307451   -10.454526  0.000000e+00
B_TIME_WALK_3B        -4.776516      0.269863   -17.699780  0.000000e+00
lambda_boxcox_cycling  0.240909      0.107725     2.236332  2.533000e-02
lambda_boxcox_driving  0.298558      0.056998     5.238066  1.622676e-07
lambda_boxcox_pt       0.545928      0.116096     4.702382  2.571439e-06
lambda_boxcox_walking  0.122389      0.054484     2.246328  2.468299e-02

In [55]:
lr_result3B = results_3B_unrestricted.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result3B.statistic=:.3g}')
print(f'{lr_result3B.threshold=:.3g}')
print(lr_result3B.message)

lr_result3B.statistic=227
lr_result3B.threshold=9.49
H0 can be rejected at level 5.0%


In [56]:
lambda_boxcox = Beta('lambda_boxcox', 1, -10, 10, 0)
boxcox_dur_walking_restricted = boxcox(dur_walking, lambda_boxcox)
boxcox_dur_cycling_restricted = boxcox(dur_cycling, lambda_boxcox)
boxcox_dur_pt_restricted = boxcox(dur_pt, lambda_boxcox)
boxcox_dur_driving_restricted = boxcox(dur_driving, lambda_boxcox)

In [57]:
V_walk_3B_restricted = (
    B_TIME_WALK_3B * boxcox_dur_walking_restricted
)

V_cycle_3B_restricted = (
    ASC_CYCLE_3B
    + B_AGE_CYCLE_3B * age
    + B_TIME_CYCLE_3B * boxcox_dur_cycling_restricted
)

V_pt_3B_restricted = (
    ASC_PT_3B
    + B_AGE_PT_3B * age
    + B_TIME_PT_3B * boxcox_dur_pt_restricted
    + B_COST_3B * cost_transit
)

V_car_3B_restricted = (
    ASC_CAR_3B
    + B_AGE_CAR_3B * age
    + B_TIME_CAR_3B * boxcox_dur_driving_restricted
    + B_COST_3B * cost_driving
)

V_3B_restricted = {1: V_walk_3B_restricted, 2: V_cycle_3B_restricted, 3: V_pt_3B_restricted, 4: V_car_3B_restricted}
logprob_3B_restricted = loglogit(V_3B_restricted, None, travel_mode)
biogeme_3B_restricted = bio.BIOGEME(database, logprob_3B_restricted)
biogeme_3B_restricted.modelName = 'model_3B_restricted'

results_3B_restricted = biogeme_3B_restricted.estimate()
print(results_3B_restricted.print_general_statistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4180.873
Final log likelihood:	-4180.873
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00287
Akaike Information Criterion:	8385.746
Bayesian Information Criterion:	8463.952
Final gradient norm:	1.3723E-01
Nbr of threads:	8



In [58]:
results_3B_restricted.getEstimatedParameters()

C:\Users\vaffs\AppData\Local\Temp\ipykernel_22552\883718923.py:1: DeprecationWarning: getEstimatedParameters is deprecated; use get_estimated_parameters instead.
  results_3B_restricted.getEstimatedParameters()


Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_3B      -0.432800      0.223297    -1.938222  5.259615e-02
ASC_CYCLE_3B    -2.177730      0.283832    -7.672609  1.687539e-14
ASC_PT_3B        1.320169      0.188531     7.002412  2.515987e-12
B_AGE_CAR_3B     0.020586      0.002522     8.162469  2.220446e-16
B_AGE_CYCLE_3B   0.004175      0.004216     0.990152  3.220999e-01
B_AGE_PT_3B      0.014157      0.002785     5.083219  3.710914e-07
B_COST_3B       -0.132956      0.015292    -8.694566  0.000000e+00
B_TIME_CAR_3B   -2.914657      0.245062   -11.893533  0.000000e+00
B_TIME_CYCLE_3B -2.897486      0.244024   -11.873798  0.000000e+00
B_TIME_PT_3B    -2.397664      0.177446   -13.512055  0.000000e+00
B_TIME_WALK_3B  -5.350993      0.249983   -21.405458  0.000000e+00
lambda_boxcox    0.198872      0.046666     4.261648  2.029249e-05

In [59]:
lr_result3B = results_3B_restricted.likelihood_ratio_test(results_pref, 0.05)
print(f'{lr_result3B.statistic=:.3g}')
print(f'{lr_result3B.threshold=:.3g}')
print(lr_result3B.message)

lr_result3B.statistic=210
lr_result3B.threshold=3.84
H0 can be rejected at level 5.0%


Testing the unrestricted model where each utility has its own lambda, with the restriction of all lambdas equal.

In [60]:
lr_result3B = results_3B_unrestricted.likelihood_ratio_test(results_3B_restricted, 0.05)
print(f'{lr_result3B.statistic=:.3g}')
print(f'{lr_result3B.threshold=:.3g}')
print(lr_result3B.message)

lr_result3B.statistic=17.1
lr_result3B.threshold=7.81
H0 can be rejected at level 5.0%


#### Power series

Trying square

In [61]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3C = Beta('ASC_CYCLE_3C', 0, None, None, 0)
ASC_PT_3C    = Beta('ASC_PT_3C', 0, None, None, 0)
ASC_CAR_3C   = Beta('ASC_CAR_3C', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3C  = Beta('B_TIME_WALK_3C', 0, None, None, 0)
B_TIME_CYCLE_3C = Beta('B_TIME_CYCLE_3C', 0, None, None, 0)
B_TIME_PT_3C    = Beta('B_TIME_PT_3C', 0, None, None, 0)
B_TIME_CAR_3C   = Beta('B_TIME_CAR_3C', 0, None, None, 0)

# Power parameters
B_TIME_WALK_3C_squared  = Beta('B_TIME_WALK_3C_squared', 0, None, None, 0)
B_TIME_CYCLE_3C_squared = Beta('B_TIME_CYCLE_3C_squared', 0, None, None, 0)
B_TIME_PT_3C_squared    = Beta('B_TIME_PT_3C_squared', 0, None, None, 0)
B_TIME_CAR_3C_squared   = Beta('B_TIME_CAR_3C_squared', 0, None, None, 0)

# Generic cost coefficient
B_COST_3C = Beta('B_COST_3C', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_3C = Beta('B_AGE_CYCLE_3C', 0, None, None, 0)
B_AGE_PT_3C = Beta('B_AGE_PT_3C', 0, None, None, 0)
B_AGE_CAR_3C = Beta('B_AGE_CAR_3C', 0, None, None, 0)

In [62]:
V_walk_3C_piecewise_squared = (
    B_TIME_WALK_3C * dur_walking
    + B_TIME_WALK_3C_squared * dur_walking * dur_walking
)

V_cycle_3C_piecewise_squared = (
    ASC_CYCLE_3C
    + B_AGE_CYCLE_3C * age
    + B_TIME_CYCLE_3C * dur_cycling
    + B_TIME_CYCLE_3C_squared * dur_cycling * dur_cycling
)

V_pt_3C_piecewise_squared = (
    ASC_PT_3C
    + B_AGE_PT_3C * age
    + B_TIME_PT_3C * dur_pt
    + B_TIME_PT_3C_squared * dur_pt * dur_pt
    + B_COST_3C * cost_transit
)

V_car_3C_piecewise_squared = (
    ASC_CAR_3C
    + B_AGE_CAR_3C * age
    + B_TIME_CAR_3C * dur_driving
    + B_TIME_CAR_3C_squared * dur_driving * dur_driving
    + B_COST_3C * cost_driving
)

V_3C_piecewise_squared ={1: V_walk_3C_piecewise_squared, 2: V_cycle_3C_piecewise_squared, 3: V_pt_3C_piecewise_squared, 4: V_car_3C_piecewise_squared}
logprob_3C_piecewise_squared = loglogit(V_3C_piecewise_squared, None, travel_mode)
biogeme_3C_piecewise_squared = bio.BIOGEME(database, logprob_3C_piecewise_squared)
biogeme_3C_piecewise_squared.modelName = 'model_3C_power_series_2'

results_3C_piecewise_squared = biogeme_3C_piecewise_squared.estimate()
print(results_3C_piecewise_squared.print_general_statistics())

Number of estimated parameters:	15
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4223.351
Final log likelihood:	-4223.351
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00355
Akaike Information Criterion:	8476.703
Bayesian Information Criterion:	8574.46
Final gradient norm:	1.0073E-01
Nbr of threads:	8



Trying cubed

In [63]:
# Alternative specific constants (ASC_walk is normalized to 0)
ASC_CYCLE_3C = Beta('ASC_CYCLE_3C', 0, None, None, 0)
ASC_PT_3C    = Beta('ASC_PT_3C', 0, None, None, 0)
ASC_CAR_3C   = Beta('ASC_CAR_3C', 0, None, None, 0)

# Alternative specific time coefficient
B_TIME_WALK_3C  = Beta('B_TIME_WALK_3C', 0, None, None, 0)
B_TIME_CYCLE_3C = Beta('B_TIME_CYCLE_3C', 0, None, None, 0)
B_TIME_PT_3C    = Beta('B_TIME_PT_3C', 0, None, None, 0)
B_TIME_CAR_3C   = Beta('B_TIME_CAR_3C', 0, None, None, 0)

# Power parameters
B_TIME_WALK_3C_cubed  = Beta('B_TIME_WALK_3C_cubed', 0, None, None, 0)
B_TIME_CYCLE_3C_cubed = Beta('B_TIME_CYCLE_3C_cubed', 0, None, None, 0)
B_TIME_PT_3C_cubed    = Beta('B_TIME_PT_3C_cubed', 0, None, None, 0)
B_TIME_CAR_3C_cubed   = Beta('B_TIME_CAR_3C_cubed', 0, None, None, 0)

# Generic cost coefficient
B_COST_3C = Beta('B_COST_3C', 0, None, None, 0)

age = Variable('age')
B_AGE_CYCLE_3C = Beta('B_AGE_CYCLE_3C', 0, None, None, 0)
B_AGE_PT_3C = Beta('B_AGE_PT_3C', 0, None, None, 0)
B_AGE_CAR_3C = Beta('B_AGE_CAR_3C', 0, None, None, 0)

In [64]:
V_walk_3C_piecewise_cubed = (
    B_TIME_WALK_3C * dur_walking
    + B_TIME_WALK_3C_cubed * dur_walking * dur_walking * dur_walking
)

V_cycle_3C_piecewise_cubed = (
    ASC_CYCLE_3C
    + B_AGE_CYCLE_3C * age
    + B_TIME_CYCLE_3C * dur_cycling
    + B_TIME_CYCLE_3C_cubed * dur_cycling * dur_cycling * dur_cycling
)

V_pt_3C_piecewise_cubed = (
    ASC_PT_3C
    + B_AGE_PT_3C * age
    + B_TIME_PT_3C * dur_pt
    + B_TIME_PT_3C_cubed * dur_pt * dur_pt * dur_pt
    + B_COST_3C * cost_transit
)

V_car_3C_piecewise_cubed = (
    ASC_CAR_3C
    + B_AGE_CAR_3C * age
    + B_TIME_CAR_3C * dur_driving
    + B_TIME_CAR_3C_cubed * dur_driving * dur_driving * dur_driving
    + B_COST_3C * cost_driving
)

V_3C_piecewise_cubed ={1: V_walk_3C_piecewise_cubed, 2: V_cycle_3C_piecewise_cubed, 3: V_pt_3C_piecewise_cubed, 4: V_car_3C_piecewise_cubed}
logprob_3C_piecewise_cubed = loglogit(V_3C_piecewise_cubed, None, travel_mode)
biogeme_3C_piecewise_cubed = bio.BIOGEME(database, logprob_3C_piecewise_cubed)
biogeme_3C_piecewise_cubed.modelName = 'model_3C_power_series_3'

results_3C_piecewise_cubed = biogeme_3C_piecewise_cubed.estimate()
print(results_3C_piecewise_cubed.print_general_statistics())

Number of estimated parameters:	15
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4242.161
Final log likelihood:	-4242.161
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00354
Akaike Information Criterion:	8514.322
Bayesian Information Criterion:	8612.08
Final gradient norm:	4.9486E-02
Nbr of threads:	8



In [65]:
V_pref = V_3B_unrestricted
results_pref = results_3B_unrestricted

## Model 4

In [66]:
mu_motorized = Beta('mu_motorized', 1, 0, None, 0)
motorized = OneNestForNestedLogit(nest_param=mu_motorized, list_of_alternatives=[3, 4], name='motorized')
walk = OneNestForNestedLogit(nest_param=1, list_of_alternatives=[1], name='walk')
bike = OneNestForNestedLogit(nest_param=1, list_of_alternatives=[2], name='bike')
nests = NestsForNestedLogit(choice_set=V_pref, tuple_of_nests=(motorized, walk, bike))

In [67]:
logprob_4 = lognested(V_pref, None, nests, travel_mode)
biogeme_4 = bio.BIOGEME(database, logprob_4)
biogeme_4.modelName = 'model_4'
results_4 = biogeme_4.estimate()
print(results_4.print_general_statistics())

Number of estimated parameters:	16
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4169.65
Final log likelihood:	-4169.65
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00384
Akaike Information Criterion:	8371.299
Bayesian Information Criterion:	8475.574
Final gradient norm:	4.3523E-01
Nbr of threads:	8



In [68]:
table_4 = results_4.get_estimated_parameters()
display(table_4)


Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_CAR_3B             0.238685      0.628654     0.379676  7.041857e-01
ASC_CYCLE_3B          -1.818140      0.461374    -3.940709  8.124120e-05
ASC_PT_3B              1.490261      0.367033     4.060286  4.901267e-05
B_AGE_CAR_3B           0.019911      0.002473     8.051089  8.881784e-16
B_AGE_CYCLE_3B         0.004272      0.004160     1.026929  3.044537e-01
B_AGE_PT_3B            0.015685      0.002683     5.845105  5.062496e-09
B_COST_3B             -0.084276      0.023830    -3.536498  4.054692e-04
B_TIME_CAR_3B         -2.246804      0.561608    -4.000663  6.316539e-05
B_TIME_CYCLE_3B       -2.248890      0.555023    -4.051889  5.080571e-05
B_TIME_PT_3B          -2.237772      0.498782    -4.486476  7.241101e-06
B_TIME_WALK_3B        -4.278765      0.365833   -11.695961  0.000000e+00
lambda_boxcox_cycling  0.176629      0.140828     1.254216  2.097633e-01
lambda_boxcox_driving  0.208582      0.073734     2.828848  4.671582e-03
lambda_boxcox_pt       0.442208      0.119093     3.713119  2.047208e-04
lambda_boxcox_walking  0.084720      0.060885     1.391467  1.640837e-01
mu_motorized           1.448713      0.299572     4.835951  1.325108e-06

The nested parameter is greater than one, consistently with the theory.

In [69]:
lr_test_existing = results_4.likelihood_ratio_test(
    results_pref, significance_level=0.05
)
print(f'{lr_test_existing.statistic=:.3g}')
print(f'{lr_test_existing.threshold=:.3g}')
print(lr_test_existing.message)


lr_test_existing.statistic=5.31
lr_test_existing.threshold=3.84
H0 can be rejected at level 5.0%


## Market shares

In [70]:
def get_weights(df):
    # Size of each segment in the population (N_g)
    census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }

    # Size of the population (N)
    population_size = sum(census.values())

    # Identify each segment in the database
    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Size of each segment in the sample (S_g)
    sample_segments = {
        segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
    }

    # Size of the sample (S)
    sample_size = sum(sample_segments.values())

    # Weights (omega_g = (N_g / N) * (S / S_g))
    weights = {
        segment_name: census[segment_name] * sample_size / (segment_size * population_size)
        for segment_name, segment_size in sample_segments.items()
    }

    return weights

def get_segments_size_in_sample(df):
    # Identify each segment in the database
    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Size of each segment in the sample (S_g)
    sample_segments = {
        segment_name: segment_rows.sum() for segment_name, segment_rows in filters.items()
    }

    return sample_segments


def get_market_shares(df, V, alpha = 0.9):
    """
    Input parameters :
    -> database : a panda dataframe,
    -> V : a dictionnary with the utility functions
       (1:walk, 2:cycle, 3:pt, 4:car)
    -> alpha : size of the confidence intervals
    Output :
    -> market_shares : dictionnary with the market shares 
        (1:walk, 2:cycle, 3:pt, 4:car)
    -> left : dict with the left bound of IC
    -> right : dict with the right bound of IC
    The three dicts are in %
    """
    weights = get_weights(df)

    filters = {
        'female_44_less': (df.age <= 44) & (df.female == 1),
        'female_45_more': (df.age >= 45) & (df.female == 1),
        'male_44_less': (df.age <= 44) & (df.female == 0),
        'male_45_more': (df.age >= 45) & (df.female == 0),
    }

    # Create a new dataframe to add the market shares in it
    df_ms = df.copy()

    # Add the weights in the dataframe
    for segment_name, segment_rows in filters.items():
        df_ms.loc[segment_rows, 'weight'] = weights[segment_name]

    # Useful variables for later
    weight = Variable('weight')
    travel_mode = Variable('travel_mode')

    # Apply model to the new dataframe
    database_ms = db.Database('lpcm_ms', df_ms)
    logprob_ms  = loglogit(V, None, travel_mode)
    biogeme_ms  = bio.BIOGEME(database_ms, logprob_ms)
    biogeme_ms.modelName = 'market_shares'
    results_ms  = biogeme_ms.estimate()

    # Normalization for probabilities
    normV = exp(V[1]) + exp(V[2]) + exp(V[3]) + exp(V[4])

    # Probabilities
    prob_walk  = exp(V[1]) / normV
    prob_cycle = exp(V[2]) / normV
    prob_pt    = exp(V[3]) / normV
    prob_car   = exp(V[4]) / normV

    # Computation of interesting formulas
    simulate = {
        'weight': weight,
        'Prob. walk': prob_walk,
        'Prob. cycle': prob_cycle,
        'Prob. PT': prob_pt,
        'Prob. car': prob_car
    }
    biosim_ms = bio.BIOGEME(database_ms, simulate)
    simulated_values = biosim_ms.simulate(results_ms.get_beta_values())

    simulated_values['Weighted walk'] = (
        simulated_values['weight'] * simulated_values['Prob. walk']
    )
    simulated_values['Weighted cycle'] = (
        simulated_values['weight'] * simulated_values['Prob. cycle']
    )
    simulated_values['Weighted PT'] = (
        simulated_values['weight'] * simulated_values['Prob. PT']
    )
    simulated_values['Weighted car'] = (
        simulated_values['weight'] * simulated_values['Prob. car']
    )

    # Market shares
    market_share_walk  = simulated_values['Weighted walk'].mean()
    market_share_cycle = simulated_values['Weighted cycle'].mean()
    market_share_pt    = simulated_values['Weighted PT'].mean()
    market_share_car   = simulated_values['Weighted car'].mean()

    market_shares = {
        1: market_share_walk*100, 
        2: market_share_cycle*100,
        3: market_share_pt*100,
        4: market_share_car*100,
    }

    # Confidence intervals
    biogeme_ms.bootstrap_samples = 100
    results_bootstrapping = biogeme_ms.estimate(run_bootstrap=True)
    betas = biogeme_ms.free_beta_names
    b = results_bootstrapping.get_betas_for_sensitivity_analysis(betas)
    left, right = biosim_ms.confidence_intervals(b, alpha)

    left['Weighted walk']  = left['weight'] * left['Prob. walk']
    left['Weighted cycle'] = left['weight'] * left['Prob. cycle']
    left['Weighted PT']    = left['weight'] * left['Prob. PT']
    left['Weighted car']   = left['weight'] * left['Prob. car']

    right['Weighted walk']  = right['weight'] * right['Prob. walk']
    right['Weighted cycle'] = right['weight'] * right['Prob. cycle']
    right['Weighted PT']    = right['weight'] * right['Prob. PT']
    right['Weighted car']   = right['weight'] * right['Prob. car']

    left_market_share_walk  = left['Weighted walk'].mean()
    left_market_share_cycle = left['Weighted cycle'].mean()
    left_market_share_pt    = left['Weighted PT'].mean()
    left_market_share_car   = left['Weighted car'].mean()

    right_market_share_walk  = right['Weighted walk'].mean()
    right_market_share_cycle = right['Weighted cycle'].mean()
    right_market_share_pt    = right['Weighted PT'].mean()
    right_market_share_car   = right['Weighted car'].mean()

    left = {
        1: left_market_share_walk*100,
        2: left_market_share_cycle*100,
        3: left_market_share_pt*100,
        4: left_market_share_car*100,
    }

    right = {
        1: right_market_share_walk*100,
        2: right_market_share_cycle*100,
        3: right_market_share_pt*100,
        4: right_market_share_car*100,
    }

    return market_shares, left, right

def display_market_shares(df, V, alpha = 0.9):
    market_shares, left, right = get_market_shares(df, V, alpha)
    alternative_names = {
        1: 'walk',
        2: 'cycle',
        3: 'PT',
        4: 'car'
    }
    for i in np.arange(1,5):
        print(
            f'Market share for {alternative_names[i]}: {market_shares[i]:.2f}% '
            f'CI: ['
            f'{left[i]:.2f}%-'
            f'{right[i]:.2f}'
            f'%]'
        )

def get_sample_market_shares(df):
    market_shares = {}
    # Compute the segments size in the sample
    segments = {
        'walk': sum(df.travel_mode == 1),
        'cycle' : sum(df.travel_mode == 2),
        'pt' : sum(df.travel_mode == 3),
        'car' : sum(df.travel_mode == 4)
    }
    S = sum(segments.values())
    for name, value in segments.items():
        market_shares[name] = value/S
    return market_shares

def display_sample_market_shares(df):
    market_shares = get_sample_market_shares(df)
    for name, value in market_shares.items():
        print(f'Real market share within sample for {name}: {100*value:.2f}% ')

def get_weighted_market_shares(df):
    # The predicted share for alternative i is given by W(i) = 1/S * sum_{over individuals}(omega_n * P(i,x_n))
    # Where S the number of individuals in the sample, omega the weights, and P(i,x_n) the probability
    # that individual n choses alternative i. In our case we have two possibilities.
    # 1) We can compute the market shares within the sample, without taking the weights into account (equivalently,
    #    considering all weights equal to 1, the sampling is perfectly representative of the population)
    #    (that is what the function get_sample_market_shares does)
    # 2) We can take those weights into accounts, and using the formula above where P(i,x_n) is the percentage
    #    of individuals within the segment to which individual n belongs that chose alternative i. After some manipulations
    #    of equations, we find W(i) = sum_{over segments} (N_g / N) (S_g(i) / S_g)
    #    where N_g/N is the true proportion of the segment in the population, S_g the size of the segment in the sample
    #    and S_g(i) the number of persons within this segment having chosen alternative i
    census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }
    N = sum(census.values())

    filters = {
        'female_44_less': sum((df.age <= 44) & (df.female == 1)),
        'female_45_more': sum((df.age >= 45) & (df.female == 1)),
        'male_44_less': sum((df.age <= 44) & (df.female == 0)),
        'male_45_more': sum((df.age >= 45) & (df.female == 0)),
    }
    S = sum(filters.values())

    market_shares = {}

    alternative_names = {1: 'walk', 2: 'cycle', 3: 'pt', 4: 'car'}

    for i in np.arange(1,5):
        filters_alternative = {
            'female_44_less': sum((df.age <= 44) & (df.female == 1) & (df.travel_mode == i)),
            'female_45_more': sum((df.age >= 45) & (df.female == 1) & (df.travel_mode == i)),
            'male_44_less': sum((df.age <= 44) & (df.female == 0) & (df.travel_mode == i)),
            'male_45_more': sum((df.age >= 45) & (df.female == 0) & (df.travel_mode == i)),
        }
        ms = 0
        for name in census.keys():
            ms += (census[name] / N) * (filters_alternative[name] / filters[name])
        market_shares[alternative_names[i]] = ms
    
    return market_shares

    

def display_weighted_market_shares(df):
    market_shares = get_weighted_market_shares(df)
    for name, value in market_shares.items():
        print(f'Weighted market share for {name}: {100*value:.2f}% ')

In [71]:
#########################################
# Display of relevant data for the report
#########################################

# Size of segments within sample
segments_size = get_segments_size_in_sample(df)
print('Size of segments within sample : ', segments_size)
print('S = ', sum(segments_size.values()))

# Size of segments in total population
census = {
        'female_44_less': 2841376,
        'female_45_more': 1519948,
        'male_44_less': 2926408,
        'male_45_more': 1379198,
    }
print('Size of segments within population : ', census)
print('N = ', sum(census.values()))

# Weights
print(get_weights(df))

# Sample and weighted market shares
display_sample_market_shares(df)
display_weighted_market_shares(df)

# Market shares for a few models
print('Model 1A')
display_market_shares(df, V_1A)
print('Model 2B')
display_market_shares(df, V_2A)
print('Model 3B unrestricted')
display_market_shares(df, V_3B_unrestricted)
# Add model 4 once it is implemented
# print('Model 4')
# display_market_shares(df, V_4)

Size of segments within sample :  {'female_44_less': np.int64(1673), 'female_45_more': np.int64(957), 'male_44_less': np.int64(1432), 'male_45_more': np.int64(938)}
S =  5000
Size of segments within population :  {'female_44_less': 2841376, 'female_45_more': 1519948, 'male_44_less': 2926408, 'male_45_more': 1379198}
N =  8666930
{'female_44_less': np.float64(0.9798001063863486), 'female_45_more': np.float64(0.9162658659077806), 'male_44_less': np.float64(1.178953219644437), 'male_45_more': np.float64(0.8482590381781011)}
Real market share within sample for walk: 18.16% 
Real market share within sample for cycle: 3.18% 
Real market share within sample for pt: 34.74% 
Real market share within sample for car: 43.92% 
Weighted market share for walk: 18.28% 
Weighted market share for cycle: 3.33% 
Weighted market share for pt: 35.06% 
Weighted market share for car: 43.33% 
Model 1A


100%|██████████| 100/100 [00:34<00:00,  2.86it/s]


Market share for walk: 18.10% CI: [17.06%-19.19%]
Market share for cycle: 3.18% CI: [2.61%-3.71%]
Market share for PT: 34.96% CI: [33.27%-36.87%]
Market share for car: 43.76% CI: [41.77%-45.66%]
Model 2B


100%|██████████| 100/100 [01:37<00:00,  1.03it/s]


Market share for walk: 18.32% CI: [17.06%-19.44%]
Market share for cycle: 3.23% CI: [2.59%-3.92%]
Market share for PT: 35.01% CI: [33.02%-37.14%]
Market share for car: 43.44% CI: [41.20%-45.72%]
Model 3B unrestricted


100%|██████████| 100/100 [03:37<00:00,  2.17s/it]


Market share for walk: 18.32% CI: [17.01%-19.55%]
Market share for cycle: 3.23% CI: [2.54%-3.98%]
Market share for PT: 35.01% CI: [32.73%-37.27%]
Market share for car: 43.44% CI: [40.96%-46.04%]


## Forecasting